<a href="https://colab.research.google.com/github/haberlmatt/cdeep3m-colab/blob/master/CDeep3M_V2_Training_Model_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='#000FF'> **CDeep3M2 Guide:** </font>

Click run button of cell 1 below to install CDeep3M (expected time 20-30min.)

Make selections in cell 2 and click run button

In [0]:
#@title # <==== **CDeep3M Colab -- Installation**
#@markdown (Time est.: 20-30min)

#@markdown A hardware check will initially performed to ensure the 
#@markdown runtime envirnment has sufficient GPU vRAM, 
#@markdown otherwise please click: Runtime -> Factory Reset
#################
# Prep
#################

!mkdir -p /home/
%cd /home/
# Get the repo
!git clone https://github.com/haberlmatt/cdeep3m-colab.git

# Install CDeep3M - Packaged into a installer file
!chmod 777 /home/cdeep3m-colab/*

# Make sure enough GPU memory available on this instance
!python /home/cdeep3m-colab/tests/test_GPUmem.py
#!nvidia-smi

##################################
# Run CDeep3M installation
##################################
!/bin/bash /home/cdeep3m-colab/CDeep3M_V2_colab-installer.sh > /home/CDeep3m_installation.log

#################
# Set Environment
#################
os.environ['PATH'] += ":/home/nd_sense/caffe_nd_sense_segmentation/"
os.environ['CAFFE_PATH'] = "/home/nd_sense/caffe_nd_sense_segmentation/"
os.environ['PYTHONPATH'] = "/home/nd_sense/caffe_nd_sense_segmentation/distribute/python/"
os.environ['PATH'] += ":/home/cdeep3m/"
from IPython.display import clear_output
clear_output()
print("CDeep3M installation completed \n   Runnning CDeep3M Version")
!cat /home/cdeep3m/VERSION
print("\nPlease see https://github.com/haberlmatt/cdeep3m-colab for further information on using CDeep3M")

In [0]:
#@title # **CDeep3M Colab -- Train model from scratch GUI**
#@markdown <font color='#FF000'>(**NOTE:** Colab is not recommended to train a model from scratch, since the runtimes are too long to achieve good results, consider retraining from another model or running the CDeep3M docker container using a local GPU to train a model from scratch)</font>
#@markdown ### See the CDeep3M model zoo: http://www.cellimagelibrary.org/cdeep3m
#@markdown Please make sure to run installation above first

#@markdown ### Insert Path to your images here:
Image_Path = '/home/cdeep3m/mito_testsample/training/images' #@param {type:"string"}
#@markdown ### Insert Path to your labels here:
Label_Path = '/home/cdeep3m/mito_testsample/training/labels' #@param {type:"string"}

secondary_augm=int(-1)
tertiary_augm=0
# Hidden feature, uncomment following two lines to set secondary and tertiary augmentations:
#secondary_augm = -1 #@param {type:"slider", min:-1, max:10, step:1}
#tertiary_augm = 0 #@param {type:"slider", min:0, max:10, step:1}


#@markdown ### How many training iterations:
num_iter = 10000 #@param {type:"slider", min:5000, max:50000, step:1000}
#@markdown ### Train network seeing 1 frame, 3 frames, 5 frames:
train_1fm = True #@param {type:"boolean"}
train_3fm = True #@param {type:"boolean"}
train_5fm = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Enter a path where trained model will be written:
output_path = "/home/trainedmodel/" #@param {type:"string"}
#@markdown ---

# Convert into 1fm, 3fm and 5fm selection into string
fm_list = [str('1fm' * train_1fm), str('3fm' * train_3fm), str('5fm' * train_5fm)]
filtered_fm = filter(lambda x: x != "", fm_list)
combined_fm = list(filtered_fm)
fm_string = ','.join(combined_fm)
if fm_string is not None and fm_string !='':
    print('Running: ' + fm_string)
else:
    print('Provide a selection for which fm model to run')

import os
print('Starting to run ...')
# Preprocess Training Data
augmented_path='/home/augmented_data/'
exec_command=str('python3 PreprocessTraining.py ' + Image_Path + ' ' + Label_Path + ' ' + str(secondary_augm) + ' ' + str(tertiary_augm) + ' ' + augmented_path) 
print(exec_command)
!{exec_command}
# currently removed --models ' + fm_string + ' '
exec_command=str('runtraining.sh ' + '--numiterations ' + str(num_iter)  + ' ' + augmented_path +  ' ' + output_path) 
print(exec_command)

!{exec_command}